In [1]:
# %pip install openai-agents

### Running agents

In [3]:
from agents import Agent, Runner

agent = Agent(
    name="Assistant", 
    instructions="You are a helpful assistant that can answer questions and help with tasks.",
    model="gpt-4o-mini"
    )

In [4]:
from dotenv import load_dotenv
load_dotenv()


True

In [5]:
result = await Runner.run(starting_agent=agent, input="tell me a short story")
result.final_output

'Once upon a time, in a quaint little village nestled between two hills, there lived a curious girl named Lila. Every day after school, she explored the nearby woods, making friends with the animals and gathering wildflowers.\n\nOne sunny afternoon, Lila stumbled upon a hidden glade filled with vibrant flowers and the soft hum of bees. At the center stood a majestic tree, its trunk thick and gnarled, with branches reaching high into the sky. Curious, she approached and noticed a small door carved into the bark.\n\nWith a gentle push, the door creaked open, revealing a shimmering light inside. Hesitant but excited, Lila stepped through the doorway and found herself in a magical world filled with talking animals and twinkling stars overhead.\n\nA wise old owl welcomed her and explained that this was a realm where dreams came to life. Lila shared her wish to understand the language of the animals in her village. The owl nodded and granted her wish, but with one condition: she must always 

#### Streaming responses

In [6]:
from openai.types.responses import ResponseTextDeltaEvent
response = Runner.run_streamed(
    starting_agent=agent,
    input="tell me a short story"
)
async for event in response.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Once, in a quaint village surrounded by whispering woods, lived an elderly woman named Esme. She was known for her magical garden filled with flowers that shimmered under the moonlight. Every evening, children gathered at her gate, eager to hear her tales of adventure and wonder.

One night, a curious boy named Leo approached her. "Esme, do you have a story about stars?" he asked, eyes wide with anticipation. 

With a gentle smile, Esme began, "Long ago, the stars fell from the sky, scattering across the earth. The world grew dim without their light, and the creatures of the night grew sad. A brave little firefly named Luma decided to embark on a journey to gather the stars and return them home."

As Esme spoke, the village around them transformed. The air was filled with shimmering lights as if Luma’s journey was unfolding before their eyes. They envisioned Luma flying through dark forests, navigating rivers, and facing fierce winds, all the while collecting the fallen stars, each one

#### Using tools

In [7]:
from agents import function_tool

@function_tool
def multiply(x: float, y: float) -> float:
    """Multiply two floating numbers and return the result."""
    return x * y

In [8]:
agent = Agent(
    name="Assistant",
    instructions="You're a helpful assistant, remember to always use the provided tools whenever possible. Do not rely on your own knowledge too much and instead use your tools to answer queries.",
    model="gpt-4o-mini",
    tools=[multiply],
)

In [7]:
from openai.types.responses import (
    ResponseFunctionCallArgumentsDeltaEvent,
    # ResponseCreatedEvent
)
response = Runner.run_streamed(
    starting_agent=agent,
    input="What is 7.814 times 103.892?"
)

# async for event in response.stream_events():
#     print(event)

async for event in response.stream_events():
    if event.type == "raw_response_event":
        if isinstance(event.data, ResponseFunctionCallArgumentsDeltaEvent):
            print(event.data.delta, end="", flush=True) # this is streamed parameters for our tool call
        elif isinstance(event.data, ResponseTextDeltaEvent):
            print(event.data.delta, end="", flush=True) # this is streamed final answer tokens
    
    elif event.type == "agent_updated_stream_event":
        # this tells us which agent is currently in use
        print(f"> Current Agent: {event.new_agent.name}")
    
    elif event.type == "run_item_stream_event":
        # this contains the information that we'd typically stream out to the user
        if event.name == "tool_called":
            # this is the collection of oru _full_ tool call after our tool tokens have all been streamed
            print()
            print(f"> Tool Called, name: {event.item.raw_item.name}")
            print(f"> Tool Called, args: {event.item.raw_item.arguments}")
        elif event.name == "tool_output":
            # this is the response from our tool execution
            print(f"> Tool Output: {event.item.raw_item['output']}")

> Current Agent: Assistant


{"x":7.814,"y":103.892}
> Tool Called, name: multiply
> Tool Called, args: {"x":7.814,"y":103.892}
> Tool Output: 811.812088
The result of \( 7.814 \times 103.892 \) is approximately \( 811.812 \).

#### Guardrails

In [9]:
from pydantic import BaseModel

class GuardrailOutput(BaseModel):
    is_triggered: bool
    reasoning: str

politics_agent = Agent(
    name="Politics Agent",
    instructions="Check if user is asking you about political opinions.",
    output_type=GuardrailOutput
)

In [9]:
query = "What do you think about the labor party in the UK?"

result = await Runner.run(starting_agent=politics_agent, input=query)
result.final_output

GuardrailOutput(is_triggered=True, reasoning="The user's question directly asks for an opinion about the Labor Party in the UK, which is a political opinion request.")

In [10]:
from agents import (
    GuardrailFunctionOutput,
    RunContextWrapper,
    input_guardrail
)

@input_guardrail
async def politics_guardrail(
    context: RunContextWrapper[None],
    agent: Agent,
    input: str
) -> GuardrailFunctionOutput:
    response = await Runner.run(starting_agent=politics_agent, input=input)
    return GuardrailFunctionOutput(
        output_info=response.final_output,
        tripwire_triggered=response.final_output.is_triggered
    )    

In [11]:
agent = Agent(
    name="Assistant",
    instructions="You're a helpful assistant, remember to always use the provided tools whenever possible. Do not rely on your own knowledge too much and instead use your tools to answer queries.",
    model="gpt-4o-mini",
    tools=[multiply],
    input_guardrails=[politics_guardrail]
)

In [12]:
response = await Runner.run(
    starting_agent=agent,
    input="What do you think about the labor party in the UK?"
)
response.final_output

InputGuardrailTripwireTriggered: Guardrail InputGuardrail triggered tripwire

#### Conversational Agents

In [13]:
from agents import function_tool

@function_tool
def add(x: float, y: float) -> float:
    """Add two floating numbers and return the result."""
    return x + y

@function_tool
def subtract(x: float, y: float) -> float:
    """Subtract two floating numbers and return the result."""
    return x - y

@function_tool
def multiply(x: float, y: float) -> float:
    """Multiply two floating numbers and return the result."""
    return x * y

@function_tool
def divide(x: float, y: float) -> float:
    """Divide two floating numbers and return the result."""
    return x / y


In [14]:
agent = Agent(
    name="Assistant",
    instructions="You're a helpful assistant, remember to always use the provided tools whenever possible. Do not rely on your own knowledge too much and instead use your tools to answer queries.",
    model="gpt-4o-mini",
    tools=[add, subtract, multiply, divide],
    input_guardrails=[politics_guardrail]
)

In [15]:
# Initialize conversation
user_input = "start conversation"
conversation_history = []

# Main conversation loop
while user_input and user_input.lower() not in ["quit", "exit", "q", "bye"]:
    # Display and get user input
    user_input = input()
    print(f"User: {user_input}")
    
    conversation_history.append({
        "role": "user", 
        "content": user_input
    })

    # Get AI response
    response = await Runner.run(
        starting_agent=agent,
        input=conversation_history
    )
    
    # Display AI response and update history
    print(f"Assistant: {response.final_output}")
    conversation_history = response.to_input_list()

User: 
Assistant: It seems like your message didn't come through. How can I assist you today?


In [27]:
# Initialize conversation
user_input = "start conversation"
conversation_history = []

# Main conversation loop
while user_input and user_input.lower() not in ["quit", "exit", "q", "bye"]:
    # Display and get user input
    user_input = input()
    print(f"User: {user_input}")
    
    conversation_history.append({
        "role": "user", 
        "content": user_input
    })

    # response = await Runner.run(
    #     starting_agent=agent,
    #     input=conversation_history
    # )
    response = Runner.run_streamed(
        starting_agent=agent,
        input=conversation_history
    )

    async for event in response.stream_events():
        if event.type == "raw_response_event":
            if isinstance(event.data, ResponseFunctionCallArgumentsDeltaEvent):
                print(event.data.delta, end="", flush=True) # this is streamed parameters for our tool call
            elif isinstance(event.data, ResponseTextDeltaEvent):
                print(event.data.delta, end="", flush=True) # this is streamed final answer tokens
        
        elif event.type == "agent_updated_stream_event":
            # this tells us which agent is currently in use
            print(f"> Current Agent: {event.new_agent.name}")
        
        elif event.type == "run_item_stream_event":
            # this contains the information that we'd typically stream out to the user
            if event.name == "tool_called":
                # this is the collection of oru _full_ tool call after our tool tokens have all been streamed
                print()
                print(f"> Tool Called, name: {event.item.raw_item.name}")
                print(f"> Tool Called, args: {event.item.raw_item.arguments}")
            elif event.name == "tool_output":
                # this is the response from our tool execution
                print(f"> Tool Output: {event.item.raw_item['output']}")

    # Display AI response and update history
    # print(f"Assistant: {response.final_output}")
    conversation_history = response.to_input_list()



User: Hello
> Current Agent: Assistant
Hello! How can I assist you today?User: what can you do?
> Current Agent: Assistant
I can help with a variety of tasks, including:

1. Performing basic math operations (addition, subtraction, multiplication, division).
2. Answering questions or providing information on a wide range of topics.
3. Assisting with problem-solving or providing explanations.

What would you like help with today?User: I need to multiple 0.22 with 1.54
> Current Agent: Assistant
{"x":0.22,"y":1.54}
> Tool Called, name: multiply
> Tool Called, args: {"x":0.22,"y":1.54}
> Tool Output: 0.3388
The result of multiplying 0.22 by 1.54 is 0.3388.User: Thanks a ton for that
> Current Agent: Assistant
You're welcome! If you have any more questions or need further assistance, feel free to ask!User: okay
> Current Agent: Assistant
Great! Just let me know if there's anything specific you need help with.User: bye
> Current Agent: Assistant
Goodbye! Have a great day! If you need assista

#### Handoff Agents

In [16]:
from agents import function_tool

@function_tool
def add(x: float, y: float) -> float:
    """Add two floating numbers and return the result."""
    return x + y

@function_tool
def subtract(x: float, y: float) -> float:
    """Subtract two floating numbers and return the result."""
    return x - y

@function_tool
def multiply(x: float, y: float) -> float:
    """Multiply two floating numbers and return the result."""
    return x * y

@function_tool
def divide(x: float, y: float) -> float:
    """Divide two floating numbers and return the result."""
    return x / y


calculator_agent = Agent(
    name="Calculator Assistant",
    instructions="You're a helpful assistant, remember to always use the provided tools whenever possible. Do not rely on your own knowledge too much and instead use your tools to answer queries.",
    model="gpt-4o-mini",
    tools=[add, subtract, multiply, divide]
)

@function_tool
def get_weather(city: str) -> str:
    """Get the current weather temperature for a city. Always returns 35°C."""
    return f"The current temperature in {city} is 35°C"

weather_agent = Agent(
    name="Weather Assistant",
    instructions="You're a helpful weather assistant. Always use the provided weather tool to check temperatures. Do not make up weather information and rely on the tool's data. Always give the temperature in Farhrenheit.",
    model="gpt-4o-mini",
    tools=[get_weather]
)


In [17]:
from agents import Agent

main_agent = Agent(
    name="Main Agent",
    instructions=(
        "You are minerva, a bot by BCG POPX. You are always cheerful and happy to help."
        "Help the user with their questions."
        "If they ask about getather, handoff to the booking agent."
        "If they ask about calculations, handoff to the calculator agent."
    ),
    handoffs=[calculator_agent, weather_agent],
)

In [18]:
from agents import Runner

response = await Runner.run(
    starting_agent=main_agent,
    input=response.to_input_list() + [{"role": "user", "content": "What's the weather in california!"}]
)
print(response.final_output)

The current temperature in California is 35°C, which is approximately 95°F. If you need more specific weather information or forecasts, let me know!


In [ ]:
import asyncio
from dataclasses import dataclass, field
from agents import Agent, Runner, RunContextWrapper

@dataclass
class ChatContext:
    user_name: str
    history: list = field(default_factory=list)

# Set up the agent
agent = Agent[ChatContext](
    name="ChatBot",
    instructions="Be concise and friendly. Refer to previous messages if helpful.",
)

async def handle_message(user_message: str, context: ChatContext):
    context.history.append({"role": "user", "content": user_message})

    result = await Runner.run(
        starting_agent=agent,
        input=context.history,  # Feed full history to LLM
        context=context,
    )

    context.history.append({"role": "assistant", "content": result.final_output})
    return result.final_output

async def main():
    context = ChatContext(user_name="Alex")

    print("Start chatting with the bot (type 'exit' to quit):\n")
    while True:
        user_input = input("You: ")
        if user_input.lower() in ("exit", "quit"):
            print("Goodbye!")
            break

        response = await handle_message(user_input, context)
        print("Bot:", response)

# For Jupyter/Notebook
await main()


Start chatting with the bot (type 'exit' to quit):

Bot: Hi Prashant! How’s your day going?
Bot: I can chat with you, answer questions, help with information, and offer recommendations. Anything specific you need help with?
Bot: Of course, Prashant! What would you like to talk about today?
Bot: You're welcome! If there's anything else you need, just let me know.
Goodbye!


In [24]:
import asyncio
from dataclasses import dataclass, field
from agents import Agent, Runner, RunContextWrapper, function_tool

# === TOOLS ===

@function_tool
def add(x: float, y: float) -> float:
    return x + y

@function_tool
def subtract(x: float, y: float) -> float:
    return x - y

@function_tool
def multiply(x: float, y: float) -> float:
    return x * y

@function_tool
def divide(x: float, y: float) -> float:
    return x / y

@function_tool
def get_weather(city: str) -> str:
    return f"The current temperature in {city} is 35°C"

# === SPECIALIZED AGENTS ===

calculator_agent = Agent(
    name="Calculator Assistant",
    instructions=(
        "You're a helpful assistant, remember to always use the provided tools whenever possible. "
        "Do not rely on your own knowledge too much and instead use your tools to answer queries."
    ),
    model="gpt-4o-mini",
    tools=[add, subtract, multiply, divide]
)

weather_agent = Agent(
    name="Weather Assistant",
    instructions=(
        "You're a helpful weather assistant. Always use the provided weather tool to check temperatures. "
        "Do not make up weather information and rely on the tool's data. Always give the temperature in Fahrenheit."
    ),
    model="gpt-4o-mini",
    tools=[get_weather]
)

# === CONTEXT ===

@dataclass
class ChatContext:
    user_name: str
    history: list = field(default_factory=list)

# === MAIN AGENT ===


main_agent = Agent[ChatContext](
    name="Main Agent",
    instructions=(
        "You are Minerva, a bot by BCG POPX. You are always cheerful and happy to help. "
        "Help the user with their questions. "
        "If they ask about weather, hand off to the Weather Assistant. "
        "If they ask about calculations, hand off to the Calculator Assistant."
    ),
    handoffs=[calculator_agent, weather_agent],
    model="gpt-4o-mini",
)

# === MESSAGE HANDLER ===

async def handle_message(user_message: str, context: ChatContext):
    context.history.append({"role": "user", "content": user_message})

    result = await Runner.run(
        starting_agent=main_agent,
        input=context.history,
        context=context,
    )

    context.history.append({"role": "assistant", "content": result.final_output})
    return result

# === CHAT LOOP ===

async def main():
    context = ChatContext(user_name="Alex")
    print("Welcome to Minerva 🤖 (type 'exit' to quit)\n")

    while True:
        user_input = input("You: ")
        if user_input.strip().lower() in ("exit", "quit"):
            print("Minerva: Goodbye! 👋")
            break

        response = await handle_message(user_input, context)
        print("Minerva:", response)

# === RUN (for Jupyter/IPython) ===
await main()


Welcome to Minerva 🤖 (type 'exit' to quit)

Minerva: RunResult:
- Last agent: Agent(name="Main Agent", ...)
- Final output (str):
    It looks like your message got cut off! How can I assist you today? 😊
- 1 new item(s)
- 1 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)
Minerva: RunResult:
- Last agent: Agent(name="Main Agent", ...)
- Final output (str):
    It seems like you're trying to ask something. Could you please provide a bit more detail? I'm here to help! 🎉
- 1 new item(s)
- 1 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)
Minerva: Goodbye! 👋


In [41]:
context = ChatContext(user_name="Alex")
resp = await handle_message("What's the weather in San Francisco?", context)

In [42]:
log_current_tool_execution(resp)

[LOG] Handoff occurred, but target agent not found.
[LOG] Agent 'Weather Assistant' is invoking tool 'get_weather' with arguments: {"city":"San Francisco"}
[LOG] Agent 'Weather Assistant' received output: The current temperature in San Francisco is 35°C from tool call
[LOG] Agent 'Weather Assistant' responded with: "The current temperature in San Francisco is 95°F."


In [37]:
def log_current_tool_execution(run_result):
    for item in run_result.new_items:
        # Case 1: Handoff occurred
        if item.type == "handoff_call_item":
            # Get the agent name (if available)
            if hasattr(item, "target_agent"):
                print(f"[LOG] Handoff to agent: {item.target_agent.name}")
            else:
                print("[LOG] Handoff occurred, but target agent not found.")

        # Case 2: Tool is being called
        elif item.type == "tool_call_item":
            tool_name = getattr(item.raw_item, "name", "unknown_tool")
            args = getattr(item.raw_item, "arguments", "{}")
            agent_name = getattr(item.agent, "name", "unknown_agent")
            print(f"[LOG] Agent '{agent_name}' is invoking tool '{tool_name}' with arguments: {args}")

        # Case 3: Tool execution output
        elif item.type == "tool_call_output_item":
            output = getattr(item, "output", None)
            agent_name = getattr(item.agent, "name", "unknown_agent")
            print(f"[LOG] Agent '{agent_name}' received output: {output} from tool call")

        # Case 4: Assistant final message
        elif item.type == "message_output_item":
            content = (
                item.raw_item.content[0].text
                if getattr(item.raw_item, "content", None)
                else "[no message content]"
            )
            agent_name = getattr(item.agent, "name", "unknown_agent")
            print(f"[LOG] Agent '{agent_name}' responded with: \"{content}\"")


In [31]:
async for event in response.stream_events():
    if event.type == "raw_response_event":
        if isinstance(event.data, ResponseFunctionCallArgumentsDeltaEvent):
            print(event.data.delta, end="", flush=True) # this is streamed parameters for our tool call
        elif isinstance(event.data, ResponseTextDeltaEvent):
            print(event.data.delta, end="", flush=True) # this is streamed final answer tokens
    
    elif event.type == "agent_updated_stream_event":
        # this tells us which agent is currently in use
        print(f"> Current Agent: {event.new_agent.name}")
    
    elif event.type == "run_item_stream_event":
        # this contains the information that we'd typically stream out to the user
        if event.name == "tool_called":
            # this is the collection of oru _full_ tool call after our tool tokens have all been streamed
            print()
            print(f"> Tool Called, name: {event.item.raw_item.name}")
            print(f"> Tool Called, args: {event.item.raw_item.arguments}")
        elif event.name == "tool_output":
            # this is the response from our tool execution
            print(f"> Tool Output: {event.item.raw_item['output']}")

AttributeError: 'RunResult' object has no attribute 'stream_events'

In [6]:
def log_current_tool_execution(result):
    """Log tool execution details using Python's logging module."""
    for item in result.new_items:
        # Case 1: Handoff occurred
        if item.type == "handoff_call_item":
            # Get the agent name (if available)
            if hasattr(item, "target_agent"):
                print(f"Handoff to agent: {item.target_agent.name}")
            else:
                print("[WARNING] Handoff occurred, but target agent not found.")

        # Case 2: Tool is being called
        elif item.type == "tool_call_item":
            tool_name = getattr(item.raw_item, "name", "unknown_tool")
            args = getattr(item.raw_item, "arguments", "{}")
            agent_name = getattr(item.agent, "name", "unknown_agent")
            print(f"[INFO] Agent '{agent_name}' is invoking tool '{tool_name}' with arguments: {args}")

        # Case 3: Tool execution output
        elif item.type == "tool_call_output_item":
            output = getattr(item, "output", None)
            agent_name = getattr(item.agent, "name", "unknown_agent")
            print(f"[INFO] Agent '{agent_name}' received output: {output} from tool call")

        # Case 4: Assistant final message
        elif item.type == "message_output_item":
            content = (
                item.raw_item.content[0].text
                if getattr(item.raw_item, "content", None)
                else "[no message content]"
            )
            agent_name = getattr(item.agent, "name", "unknown_agent")
            print(f"[INFO] Agent '{agent_name}' responded with: \"{content}\"")


In [ ]:
# =================
# Imports
# =================
from dataclasses import dataclass, field
from agents import Agent, Runner, function_tool
from pydantic import BaseModel, Field
from typing import Dict, List, Union, Any

# =================
# Tool Functions
# =================
@function_tool
def add(x: float, y: float) -> float:
    print("Adding", x, y)
    return x + y

@function_tool
def subtract(x: float, y: float) -> float:
    print("Subtracting", x, y)
    return x - y

@function_tool
def multiply(x: float, y: float) -> float:
    print("Multiplying", x, y)
    return x * y

@function_tool
def divide(x: float, y: float) -> float:
    print("Dividing", x, y)
    return x / y

@function_tool
def get_weather(city: str) -> str:
    return f"The current temperature in {city} is 35°C"

# =================
# Response Structure
# =================
class ResponseStucture(BaseModel):
    message: str
    message_type: str # 'text' or 'checkbox'
    options: list[str]
    trigger: bool # always keep it false

# =================
# Agent Definitions
# =================
calculator_agent = Agent(
    name="Calculator Assistant",
    instructions=(
        "You're a helpful assistant, remember to always use the provided tools whenever possible. "
        "Do not rely on your own knowledge too much and instead use your tools to answer queries."
    ),
    model="gpt-4o-mini",
    tools=[add, subtract, multiply, divide],
)

weather_agent = Agent(
    name="Weather Assistant",
    instructions=(
        "You're a helpful weather assistant. Always use the provided weather tool to check temperatures. "
        "Do not make up weather information and rely on the tool's data."
    ),
    model="gpt-4o-mini",
    tools=[get_weather],
)

# =================
# Data Models
# =================
@dataclass
class ChatContext:
    user_name: str
    history: list = field(default_factory=list)

# =================
# Main Agent Setup
# =================

main_agent = Agent[ChatContext](
    name="Main Agent",
    instructions=(
        "You are Novak, a bot by BCG POPX. You are always cheerful and happy to help. "
        "Help the user with their questions."
        "You have access to two tools: Calculator and Weather."
        # "If they ask about weather calculations, hand off to the Weather Assistant first then handoff that result to Calculator Assistant."
        "Whenever you don't understand the user's question, ask questions back to understand the user's question better. And try to break it down to solve it."
        "Try to solve the problem with combination of tools available. Check if result of one tool can be used as input for another tool."

        "Response Stucture: You can give output in either only text or checkbox (If you want user to select from options) format. message_type must be 'text' or 'checkbox'. Use 'options' array to specify the options for the checkbox."
    ),
    model="gpt-4o-mini",
    output_type=ResponseStucture,
    tools=[
            calculator_agent.as_tool(tool_name="calculator", tool_description="Tool to perform calculations"), 
            weather_agent.as_tool(tool_name="weather", tool_description="Tool to get weather information")
        ]
    # handoffs=[calculator_agent]
)

# =================
# Message Handler
# =================
async def handle_message(user_message: str, context: ChatContext):
    context.history.append({"role": "user", "content": user_message})

    result = await Runner.run(
        starting_agent=main_agent,
        input=context.history,
        context=context
    )
    log_current_tool_execution(result)
    context.history.append({"role": "assistant", "content": str(result.final_output)})

    return result

In [11]:
context = ChatContext(user_name="Testing")

response = await handle_message("I need to do a weather calculation. I need to multiply the temperature of San Francisco by 0.00234", context)
# dict(response.final_output)
print(response.final_output)

Multiplying 35.0 0.00234
[INFO] Agent 'Main Agent' is invoking tool 'weather' with arguments: {"input":"San Francisco"}
[INFO] Agent 'Main Agent' received output: The current temperature in San Francisco is 35°C. Is there anything else you would like to know? from tool call
[WARNING] Handoff occurred, but target agent not found.
[INFO] Agent 'Calculator Assistant' is invoking tool 'multiply' with arguments: {"x":35,"y":0.00234}
[INFO] Agent 'Calculator Assistant' received output: 0.0819 from tool call
[INFO] Agent 'Calculator Assistant' responded with: "The result of multiplying the temperature of San Francisco (35°C) by 0.00234 is approximately **0.0819**."
The result of multiplying the temperature of San Francisco (35°C) by 0.00234 is approximately **0.0819**.


In [50]:
context.history

[{'role': 'user', 'content': 'hello there I am prashant'},
 {'role': 'assistant',
  'content': "message='Hello Prashant! 😊 How can I assist you today?' message_type='text' options=[] trigger=False"},
 {'role': 'user', 'content': 'What is your name?'},
 {'role': 'assistant',
  'content': "message='I’m Novak, your cheerful assistant! How can I help you today?' message_type='text' options=[] trigger=False"},
 {'role': 'user', 'content': 'I need to solve a problem'},
 {'role': 'assistant',
  'content': "message='What problem would you like to solve? If you can share some details, I’d be happy to help!' message_type='text' options=[] trigger=False"},
 {'role': 'user', 'content': 'Can you help me?'},
 {'role': 'assistant',
  'content': "message='I can definitely help you! Here are two things I found:\\n1. The result of 2 + 2 is **4**.\\n2. The current temperature in New York is **35°C**. \\n\\nWhat else do you need assistance with?' message_type='text' options=[] trigger=False"},
 {'role': '

In [47]:
context = ChatContext(user_name="Prashant")

user_message = None
while user_message not in ["exit", "quit"]:
    user_message = input("Enter your message: ")
    response = await handle_message(user_message, context)
    print(dict(response.final_output))
    # print(response.final_output)

[INFO] Agent 'Main Agent' responded with: "{"message":"Hello Prashant! 😊 How can I assist you today?","message_type":"text","options":[],"trigger":false}"
{'message': 'Hello Prashant! 😊 How can I assist you today?', 'message_type': 'text', 'options': [], 'trigger': False}
[INFO] Agent 'Main Agent' responded with: "{"message":"I’m Novak, your cheerful assistant! How can I help you today?","message_type":"text","options":[],"trigger":false}"
{'message': 'I’m Novak, your cheerful assistant! How can I help you today?', 'message_type': 'text', 'options': [], 'trigger': False}
Adding 1.0 1.0
[INFO] Agent 'Main Agent' is invoking tool 'calculator' with arguments: {"input":"1 + 1"}
[INFO] Agent 'Main Agent' received output: The result of \( 1 + 1 \) is \( 2 \). from tool call
[INFO] Agent 'Main Agent' responded with: "{"message":"What problem would you like to solve? If you can share some details, I’d be happy to help!","message_type":"text","options":[],"trigger":false}"
{'message': 'What pro

In [ ]:
{"matches": List[str]}

In [23]:
from llm import GPT

llm = GPT()
import json

response = llm.get_response(
    model="gpt-4o-2024-08-06",
    input=[
        {"role": "user", "content": "Give me an mcq on machine learning"}
    ],
    text={
        "format": {
            "type": "json_schema",
            "name": "essay",
            "schema": {
                "type": "object",
                "properties": {
                    "message": {"type": "string"},
                    "message_type": {
                        "type": "string",
                        "enum": ["text", "checkbox", "radio", "dropdown"]
                    },
                    "options": {
                        "type": "array",
                        "items": {"type": "string"}
                    }
                },
                "required": ["message", "message_type", "options"],
                "additionalProperties": False
            },
            "strict": True
        }
    }
)

response_json = json.loads(response.output_text)

In [35]:
import os
import json
import pandas as pd
from typing import Any, Dict, List

from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()


class GPT:
    """
    A wrapper class for interacting with OpenAI's GPT models.

    Attributes:
        api_key (str): The API key for authentication with OpenAI.
        organization_key (str, optional): The organization ID for OpenAI.
        client (OpenAI): The OpenAI client instance.
    """

    def __init__(
        self,
        api_key: str = None,
        organization_key: str = None
    ) -> None:
        """
        Initializes the GPT class with OpenAI credentials.
        Args:
            api_key (str, optional): OpenAI API key. Defaults to None, in which case
                it is retrieved from environment variables.
            organization_key (str, optional): OpenAI organization ID. Defaults to None,
                retrieved from environment variables.

        Raises:
            ValueError: If no API key is provided.
        """
        api_key = os.getenv("OPENAI_API_KEY", api_key)
        organization_key = os.getenv("OPENAI_ORGANIZATION_ID", organization_key)

        if not api_key:
            raise ValueError(
                "An API key must be provided either as an argument or in the "
                "environment variable 'OPENAI_API_KEY'. This is the recommended approach."
            )

        self.api_key = api_key
        self.organization_key = organization_key
        self.client = OpenAI(
            api_key=self.api_key,
            organization=self.organization_key
        )

    def get_response(self, **kwargs: Dict[str, Any]) -> Any:
        """Sends a request to the OpenAI API and returns the response."""
        response = self.client.responses.create(**kwargs)
        return response


llm = GPT()

def get_findings(data: pd.DataFrame, client_name: str, peer_list: List[str], analysis_type: str) -> Dict[str, str]:
    """
    Get insights and findings from data analysis.

    Args:
        data (pd.DataFrame): Input data to analyze
        client_name (str): Name of the client company
        peer_list (List[str]): List of peer companies to compare against
        analysis_type (str): Type of analysis to perform

    Returns:
        Dict[str, str]: Dictionary containing findings from the analysis
    """
    findings_prompt = """I will provide you with a pandas DataFrame and exactly one ANALYSIS_TYPE from the list below. You'll also be given the CLIENT company name and a list of PEERS.
Analysis types (one-liner):
Manager Title Ratio: % of managers in each company
Support Function Analysis: % and breakdown of support functions
Compensation Distribution: compensation medians or ranges across levels
Interview Experience: candidate interview-experience scores (higher = better)
Diversity in Management: % female managers
Culture & Values Rating: Glassdoor rating for culture & values

Your task:
Identify 2–3 key insights, focusing on how CLIENT compares to its PEERS in the given analysis.
Use specific metrics or percentages to illustrate each point.
Write your findings in 2–3 concise sentences.

Placeholders:
DATAFRAME: {input_data} (e.g., df)
ANALYSIS_TYPE: {analysis_type} one of the six types above (only one per prompt)
CLIENT: {client_name} the focal company
PEERS: {peers_list} a list of peer companies ( if not provided refer to the data frame )"""
    
    response = llm.get_response(
        # model="gpt-4o-mini-2024-07-18",
        # model="gpt-4.1-nano",
        # model="gpt-4.1-nano",
        model="o4-mini",
        input=[
            {"role": "user", "content": findings_prompt.format(
                input_data=data,
                analysis_type=analysis_type,
                client_name=client_name,
                peers_list=peer_list
            )}
        ],
        text={
            "format": {
                "type": "json_schema",
                "name": "findings",
                "schema": {
                    "type": "object",
                    "properties": {
                        "findings": {
                            "type": "string",
                            "description": "A single paragraph of findings."
                        }
                    },
                    "required": ["findings"],
                    "additionalProperties": False
                },
                "strict": True
            }
        }
    )

    return json.loads(response.output_text)

In [28]:
sample_data = {
    "Company": [
        "Archer Daniels Midland",
        "Nike",
        "PepsiCo",
        "Procter & Gamble",
        "Tyson Foods"
    ],
    "CXO": [0.1, 0.1, 0.4, 2.2, 0.0],
    "Vice President": [1.1, 0.2, 0.6, 0.8, 1.1],
    "Director": [7.9, 7.3, 8.9, 12.4, 14.6],
    "Manager": [29.7, 26.0, 37.9, 41.5, 41.6]
}

In [26]:
%time
findings = get_findings(data=sample_data, client_name="PepsiCo", peer_list=["Archer Daniels Midland", "Nike", "Procter & Gamble", "Tyson Foods"], analysis_type="Manager Title Ratio")

CPU times: user 2 μs, sys: 0 ns, total: 2 μs
Wall time: 5.25 μs


In [30]:
%time
findings = get_findings(data=sample_data, client_name="PepsiCo", peer_list=["Archer Daniels Midland", "Nike", "Procter & Gamble", "Tyson Foods"], analysis_type="Manager Title Ratio")

CPU times: user 3 μs, sys: 0 ns, total: 3 μs
Wall time: 5.48 μs


In [33]:
%time
findings = get_findings(data=sample_data, client_name="PepsiCo", peer_list=["Archer Daniels Midland", "Nike", "Procter & Gamble", "Tyson Foods"], analysis_type="Manager Title Ratio")

CPU times: user 3 μs, sys: 0 ns, total: 3 μs
Wall time: 5.48 μs


In [36]:
%time
findings = get_findings(data=sample_data, client_name="PepsiCo", peer_list=["Archer Daniels Midland", "Nike", "Procter & Gamble", "Tyson Foods"], analysis_type="Manager Title Ratio")

CPU times: user 2 μs, sys: 1e+03 ns, total: 3 μs
Wall time: 5.72 μs


In [21]:
from pprint import pprint
pprint(findings["findings"])

('PepsiCo has a manager title ratio of 37.9%, placing it slightly below '
 'Procter & Gamble (41.5%) and Tyson Foods (41.6%), indicating a more '
 'centralized management structure compared to its peers. In contrast, Archer '
 'Daniels Midland and Nike have lower manager ratios at 29.7% and 26.0%, '
 "respectively, showing that PepsiCo's managerial representation is "
 'competitive yet still below the top two in its peer group.')


In [31]:
from pprint import pprint
pprint(findings["findings"])

('PepsiCo has a notably high percentage of managers at 37.9%, which is '
 "significantly above Archer Daniels Midland's 29.7% and Nike's 26.0%. "
 'However, PepsiCo is slightly lower than Procter & Gamble and Tyson Foods, '
 'which have 41.5% and 41.6% managers respectively. This positions PepsiCo in '
 'the upper mid-range for managerial representation compared to its peers.')


In [34]:
from pprint import pprint
pprint(findings["findings"])

('PepsiCo has a Manager representation of 37.9%, which is higher than Archer '
 'Daniels Midland (29.7%) and Nike (26.0%), but slightly lower than Procter & '
 'Gamble (41.5%) and Tyson Foods (41.6%). The overall manager ratio at PepsiCo '
 'indicates a comparatively balanced management structure, positioning it '
 'above some peers but below the two largest management ratios in the sample.')


In [37]:
from pprint import pprint
pprint(findings["findings"])

('PepsiCo’s management layer comprises 37.9% of its workforce, ranking third '
 'among its peers and sitting 3.6 points below the highest ratios at Procter & '
 'Gamble (41.5%) and Tyson Foods (41.6%). This is substantially higher than '
 'Archer Daniels Midland (29.7%) and Nike (26.0%), and about 3.2 points above '
 'the peer average of 34.7%, indicating a stronger mid‐management presence '
 'relative to most competitors.')
